# README.md
#get baseline T and PSA dates and values for the 270 pts 

## Table of Contents
* [Read data, clean up and get stats](#first-bullet)
* [Get the baseline T dates and values](#second-bullet)
* [Get the baseline PSA dates and values](#third-bullet)

In [38]:
import numpy as np
import pandas as pd
import sys, getopt
import csv
from datetime import datetime
import itertools

from matplotlib import pyplot
from pylab import *
import matplotlib.pyplot as plt
%matplotlib inline  
import seaborn as sns

# Read data, clean up and get stats <a class="anchor" id="first-bullet"></a>

In [39]:
df1=pd.read_excel("./data/270pts_cohort.xlsx", header=0)
list_pt=df1["person_id"].to_list()
df1.fillna('', inplace=True)
print(shape(df1))
print("")

(270, 4)



In [40]:
list(df1.columns.values)

['person_id', 'ADT_start', 'ADT_end', 'ADT_duration (days)']

In [41]:
df_tvalue=pd.read_excel("./data/pca_tresults_20210119_BP_fixed.xlsx", header=0)

df1.fillna('', inplace=True)
df1.replace(r'', np.NaN)
df1[:3]

df_tvalue.fillna('', inplace=True)
df_tvalue.replace(r'', np.NaN)

df_tvalue_grouped=df_tvalue.groupby(["person_id"]).size().reset_index(name='counts')
list_pt_ttest=df_tvalue_grouped["person_id"].to_list()
print("list of pt ids with t test results")
print(len(list_pt_ttest))

print("overlapping with t test")
overlap_list1=[x for x in list_pt if x in  list_pt_ttest]
print(len(overlap_list1))

df_psa=pd.read_csv("./data/_SELECT_g_person_id_g_psa_date_g_psa_value_FROM_dev_precisionmed_202103111416.csv", header=0) 
df_psa.fillna('', inplace=True)
df_psa.replace(r'', np.NaN)

df_psa_grouped=df_psa.groupby(["person_id"]).size().reset_index(name='counts')
list_pt_psa=df_psa_grouped["person_id"].to_list()
overlap_list2=[x for x in list_pt if x in list_pt_psa]
print("overlapping with psa test")
print(len(overlap_list2))

list of pt ids with t test results
4490
overlapping with t test
270
overlapping with psa test
268


In [42]:
df2=df_tvalue

def closest(lst, K): 
      return lst[min(range(len(lst)), key = lambda i: abs(lst[i] - K))] 

In [43]:
df1[:5]

,person_id,ADT_start,ADT_end,ADT_duration (days)
0,2,2009-02-26,2009-09-15,201
1,13,2010-06-17,2015-03-17,1734
2,25,2013-11-01,2014-02-18,109
3,38,2015-11-18,2016-07-18,243
4,81,2014-04-15,2016-06-09,786


In [44]:
list_pt

[2,
 13,
 25,
 38,
 81,
 110,
 119,
 780,
 870,
 905,
 1257,
 1503,
 1508,
 1583,
 1704,
 2061,
 2063,
 2200,
 2278,
 2605,
 2623,
 2652,
 2655,
 2660,
 3030,
 3129,
 3131,
 3155,
 3579,
 3905,
 3993,
 4056,
 4235,
 4257,
 4319,
 4595,
 4717,
 4793,
 4854,
 5021,
 5041,
 5043,
 5394,
 5561,
 5652,
 5791,
 5870,
 5873,
 5875,
 6078,
 6104,
 6268,
 6505,
 6794,
 6841,
 7236,
 7268,
 7611,
 7612,
 7798,
 7870,
 8020,
 8194,
 8272,
 8302,
 8544,
 8717,
 8758,
 9124,
 9232,
 9233,
 9474,
 9541,
 9972,
 9975,
 10124,
 10471,
 10945,
 10995,
 11482,
 11543,
 11613,
 11821,
 11859,
 11861,
 12184,
 12187,
 12208,
 12374,
 12563,
 12779,
 13170,
 13258,
 13274,
 13609,
 13613,
 13859,
 14096,
 14217,
 14221,
 14242,
 14278,
 14287,
 14486,
 14900,
 14990,
 15149,
 15174,
 15185,
 15456,
 15495,
 16175,
 16229,
 16546,
 16550,
 16558,
 16630,
 17341,
 17395,
 17559,
 17580,
 17638,
 17662,
 17709,
 17710,
 17759,
 17782,
 18303,
 18450,
 18801,
 18821,
 19068,
 19096,
 19541,
 19917,
 20015,
 20

# Get the baseline T dates and values <a class="anchor" id="second-bullet"></a>

In [45]:
##get the baseline T dates and values
result_baseline_list=[]
df2=df_tvalue

for y in list_pt:
    #print(y)    
    df1_pt=df1[(df1["person_id"] == y)]
    
    if(len(df1_pt.ADT_start)==0):
        a='NaN'
        status='NaN'
        tdate='NaN'
        tvalue='NaN'
        N_t_dates='NaN'
        N_t_dates_beforeADT='NaN'
            
    #if(df1_pt.ADT_start.values.any()!=""):
    if(len(df1_pt.ADT_start)!=0):
        
        a=df1_pt['ADT_start'].to_list()[0]

        df2_pt=df2[(df2["person_id"] == y)]    
        each_pt_tdates = df2_pt['result_date'].to_list()

        ##get rid of nan
        cleaned_a=[]
        cleaned_a= [x for x in each_pt_tdates if str(x) != 'NaN']
        cleaned_aa= [x for x in cleaned_a if str(x) != 'nan']
        cleaned_aaa= [x for x in cleaned_aa if str(x) != 'NaT']
        N_t_dates=len(cleaned_aaa)

        #only keep the ones before ADT start date, including on the day
        cleaned_aaaa= [x for x in cleaned_aaa if x <=a]
        start_sorted_bin=sorted(cleaned_aaaa)
        N_t_dates_beforeADT=len(cleaned_aaaa)
    
        if(len(start_sorted_bin)==0):
            tdate='NaN' 
            tvalue="NaN"
            #status means no baseline TT avail
            status="N"
        #print(tdate)
        
        if(len(start_sorted_bin)>0):
            #apply the closet function
            #tdate=closest(start_sorted_bin,a)   
            df3_pt=df2_pt[df2_pt.result_date <=a]
            
            if not df3_pt.empty:
                tdate=df3_pt['result_date'].to_list()[-1]
                tvalue=df3_pt['test_results'].to_list()[-1]
                #status means yes there are baseline TT avail
                status="Y"

    print(y, a, tdate, tvalue, status, N_t_dates, N_t_dates_beforeADT)
    result_per_pt=[y,a,tdate,tvalue, status, N_t_dates, N_t_dates_beforeADT]
    result_baseline_list.append(result_per_pt)
    #validate 

2 2009-02-26 00:00:00 2009-02-09 00:00:00 274.18 Y 82 20
13 2010-06-17 00:00:00 2009-09-18 00:00:00 23.72 Y 23 2
25 2013-11-01 00:00:00 2013-10-15 00:00:00 31.54 Y 12 1
38 2015-11-18 00:00:00 2015-11-18 00:00:00 288.37 Y 20 5
81 2014-04-15 00:00:00 2014-04-10 00:00:00 313.07 Y 12 1
110 2013-03-12 00:00:00 2013-03-12 00:00:00 28.52 Y 14 1
119 2013-07-02 00:00:00 2013-06-17 00:00:00 403.1 Y 12 2
780 2010-08-01 00:00:00 2010-03-18 00:00:00 294.68 Y 3 1
870 2017-06-14 00:00:00 2017-06-14 00:00:00 274.0 Y 10 1
905 2016-01-29 00:00:00 2015-11-25 00:00:00 20.95 Y 3 2
1257 2014-10-02 00:00:00 2014-10-02 00:00:00 234.83 Y 14 1
1503 2018-05-02 00:00:00 2018-05-02 00:00:00 148.0 Y 6 1
1508 2016-08-03 00:00:00 2016-07-20 00:00:00 757.0 Y 9 1
1583 2012-09-05 00:00:00 2012-08-22 00:00:00 238.11 Y 6 2
1704 2015-02-17 00:00:00 2015-02-17 00:00:00 0.0 Y 13 1
2061 2018-04-04 00:00:00 2018-03-02 00:00:00 187.63 Y 7 4
2063 2017-11-15 00:00:00 2016-12-20 00:00:00 349.75 Y 11 1
2200 2014-06-16 00:00:00 2014

21003 2016-08-31 00:00:00 2015-06-02 00:00:00 44.41 Y 8 2
21047 2018-08-13 00:00:00 2018-08-01 00:00:00 627.44 Y 5 1
21341 2019-01-14 00:00:00 2019-01-14 00:00:00 413.86 Y 6 1
21351 2015-02-20 00:00:00 2015-02-20 00:00:00 276.7 Y 2 1
21448 2015-06-01 00:00:00 2014-04-01 00:00:00 107.87 Y 11 1
21480 2017-01-11 00:00:00 2017-01-11 00:00:00 519.53 Y 11 1
21613 2016-12-22 00:00:00 2016-12-22 00:00:00 308.11 Y 11 1
21655 2016-02-03 00:00:00 2016-01-26 00:00:00 305.08 Y 11 2
21656 2015-06-18 00:00:00 2015-06-18 00:00:00 174.39 Y 21 1
21762 2017-09-15 00:00:00 2017-01-25 00:00:00 703.61 Y 11 6
21832 2016-02-09 00:00:00 2014-07-18 00:00:00 312.57 Y 10 8
21932 2015-09-10 00:00:00 2015-08-28 00:00:00 348.46 Y 16 4
21938 2018-04-12 00:00:00 2018-04-12 00:00:00 287.0 Y 8 1
22100 2017-08-09 00:00:00 2017-08-09 00:00:00 613.53 Y 16 4
22380 2016-08-05 00:00:00 2016-07-11 00:00:00 656.17 Y 25 8
22412 2014-07-23 00:00:00 2014-07-23 00:00:00 157.87 Y 11 1
22483 2017-12-28 00:00:00 2017-12-19 00:00:00 34

In [46]:
df_baseline=pd.DataFrame(result_baseline_list)
df_baseline.columns=(['person_id', 'inital_ADT_start_date', 'Tdate_baseline', 'Tvalue_baseline', 'Status',"N_T_dates","N_T_dates_beofreADT"])
df_baseline[:5]

,person_id,inital_ADT_start_date,Tdate_baseline,Tvalue_baseline,Status,N_T_dates,N_T_dates_beofreADT
0,2,2009-02-26,2009-02-09,274.18,Y,82,20
1,13,2010-06-17,2009-09-18,23.72,Y,23,2
2,25,2013-11-01,2013-10-15,31.54,Y,12,1
3,38,2015-11-18,2015-11-18,288.37,Y,20,5
4,81,2014-04-15,2014-04-10,313.07,Y,12,1


In [47]:
df_baseline.to_csv("./data/270pts_tbaseline.csv", encoding='utf-8', index=False)

# Get the baseline PSA dates and values <a class="anchor" id="third-bullet"></a>

In [49]:
def closest(lst, K): 
      return lst[min(range(len(lst)), key = lambda i: abs(lst[i] - K))] 
    
#psa_date=closest(start_sorted_bin,pt_dx_date)

In [53]:
# for these 270 pts, how many had psa results before ADT START dates?
#sort all psa dates; find the last before ADT start; grab the dates, and all values on that date 
values=[]
result_list=[]

for y in list_pt: 

    if y not in overlap_list2:
        count=0
        last_psa_date='NaN'
        values=['NaN']        

    if y in overlap_list2:
        df1_pt=df1[(df1["person_id"] == y)]
        df1_pt['ADT_start'] = pd.to_datetime(df1_pt['ADT_start'])
    
        if df1_pt["ADT_start"].empty:
            count=0
            last_psa_date='NaN'
            values=['NaN']        

        if not df1_pt["ADT_start"].empty:
            pt_ADTstart_date=df1_pt["ADT_start"].to_list()[0]
            df_psa_pt=df_psa[(df_psa["person_id"] == y)]
            df_psa_pt['psa_date'] = pd.to_datetime(df_psa_pt['psa_date'])
    
            df_psa_pt_before_ADTstart=df_psa_pt[(df_psa_pt.psa_date <= pt_ADTstart_date)&(df_psa_pt.psa_value.notnull())]
 
            each_pt_psa_date_before_ADTstart = df_psa_pt_before_ADTstart['psa_date'].to_list()
            sorted_each_pt_psa_before_ADTstart=sorted(each_pt_psa_date_before_ADTstart)
            count=len(sorted_each_pt_psa_before_ADTstart)
        
            if len(sorted_each_pt_psa_before_ADTstart)==0:
                last_psa_date='NaN'
                values=['NaN']
        
            if len(sorted_each_pt_psa_before_ADTstart)>0:
                last_psa_date=closest(sorted_each_pt_psa_before_ADTstart,pt_ADTstart_date)
                #pick up all values for that date
                sub_set=df_psa_pt_before_ADTstart[(df_psa_pt_before_ADTstart['psa_date']==last_psa_date)]
                
                #sort and get rid of redundancies
                each_pt_info= sorted(sub_set.psa_value.to_list())
                each_pt_dates=list(dict.fromkeys(each_pt_info))

                values=each_pt_dates
               # print(y,count,last_psa_date,values)
            
    result_per_pt=[y,pt_ADTstart_date,count,last_psa_date,values]
    result_list.append(result_per_pt)

/Users/yuqincai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/Users/yuqincai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [54]:
df_psa_results=pd.DataFrame(result_list)
df_psa_results.columns=(['person_id', 'ADT_startdate','Number_of_PSA_dates_with_PSA_values_before_ADTstart','Last_PSA_date_before_ADTstart', 'Last_PSA_values_before_ADTstart'])

In [55]:
df_psa_results.to_csv("./data/270pts_with_PSA_dates_and_values_before_ADT_start.csv",encoding='utf-8',index=False)